In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import csv
from IPython.display import Image
from IPython.display import display

In [2]:
df_postn = pd.read_csv('final_preprocessing_utf16.csv', encoding = 'UTF-16LE', sep=',')
display(df_postn.head(3))
print(len(df_postn))

,Unnamed: 0,no,xy,docid,key,id.x,created_time,message,lang,re_message
0,0,0,message.x,136529,psoriasis,105226382844778_320199574680790,2012-01-01 00:24:50,Ahora s. Feliz ao y mucha salud para tod@s!!!,english,ahora feliz mucha salud para tod
1,2,2,message.y,136530,psoriasis,105226382844778_320199574680790,2012-01-01 00:29:53,Ms piel y menos placas!!!!!!,english,piel menos placas
2,6,6,message.y,136534,psoriasis,105226382844778_320199574680790,2012-01-01 00:41:29,"muchas gracias, adios picores y hola nueva pie...",english,muchas gracias adios picores hola nueva piel


116141


In [3]:
# Importing Gensim
import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
corpus=list(df_postn['re_message'])
def tokenize(doc):
    tokens = ' '.join(re.findall(r"[\w']+", str(doc))).lower().split()
    x = [''.join(c for c in s if c not in string.punctuation) for s in tokens]
    x=' '.join(x)
    #print(x)
    stop_free = " ".join([i for i in x.lower().split() if i not in stop])
    #print(doc.lower().split())
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word,pos='n') for word in punc_free.split())
    normalized = " ".join(lemma.lemmatize(word,pos='v') for word in normalized.split())
    word = " ".join(word for word in normalized.split() if len(word)>3)
    #print(word.split())
    postag=nltk.pos_tag(word.split())
    #print(postag)
    #irlist=[',','.',':','#',';','CD','WRB','RB','PRP','...',')','(','-','``','@']
    poslist=['NN','NNP','NNS','RB','RBR','RBS','JJ','JJR','JJS']
    wordlist=['co', 'https', 'http','rt','www','ve','dont',"i'm","it's"]
    adjandn = [word for word,pos in postag if pos in poslist and word not in wordlist and len(word)>3]
    #normalized = adjandn.split()
    return ' '.join(adjandn)
doc_clean = [tokenize(doc).split() for doc in corpus]
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)
#clean("Why you are so late'''''?for knowing nothing cute")
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [4]:
import gensim
from gensim import corpora
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=20)
ldamodel = Lda(doc_term_matrix, num_topics=30, id2word = dictionary, update_every=10, chunksize=10000, passes=5)
#lda.printTopics(k)

In [5]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare(ldamodel, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)
#pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

In [7]:
#for i in range(len(df_postn)):
#    if '2017' in df_postn['created_time'][i]:
#        print(i, df_postn['created_time'][i])
#106239-116140
data_2017=pd.DataFrame(df_postn[106239:], columns = ['no','xy','docid','key','id.x','created_time','message','lang','re_message'])
display(data_2017.head(3))

,no,xy,docid,key,id.x,created_time,message,lang,re_message
106239,152610,message.x,156857,psoriasis,10264914954_10150788614399955,2017-01-01 00:05:01,"A Happy New Year to all, from everybody here a...",english,happy new year everybody psoriasis association
106240,152611,message.x,77072,rheumatoid arthritis,1644475815770431_361491914223236,2017-01-01 00:51:00,17 Truths People With Arthritis Wish Others Un...,english,truths people arthritis wish others understood
106241,152612,message.x,111257,psoriasis,1386580074926973_1783163101935333,2017-01-01 01:00:15,Have a safe and happy New Year!,english,safe happy new year


In [8]:
corpus_2017 = list(data_2017['re_message'])

In [9]:
doc_clean_2017 = [tokenize(doc).split() for doc in corpus_2017]
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary_2017 = corpora.Dictionary(doc_clean_2017)
#clean("Why you are so late'''''?for knowing nothing cute")
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix_2017 = [dictionary_2017.doc2bow(doc) for doc in doc_clean_2017]

In [10]:
import gensim
from gensim import corpora
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
#ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=20)
ldamodel_2017 = Lda(doc_term_matrix_2017, num_topics=20, id2word = dictionary_2017, update_every=10, chunksize=1000, passes=5)
#lda.printTopics(k)

In [11]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data_2017 = gensimvis.prepare(ldamodel_2017, doc_term_matrix_2017, dictionary_2017)
pyLDAvis.display(vis_data_2017)

In [12]:
#doc_clean_2017 = [tokenize(doc).split() for doc in corpus_2017]
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
#dictionary_2017 = corpora.Dictionary(doc_clean_2017)
#clean("Why you are so late'''''?for knowing nothing cute")
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#doc_term_matrix_2017 = [dictionary_2017.doc2bow(doc) for doc in doc_clean_2017]
def getTopicForQuery_2017(question):
    temp = tokenize(question).split()
    #print(temp)
    ques_vec = []
    ques_vec = dictionary_2017.doc2bow(temp)
    topic_vec = []
    topic_vec = ldamodel_2017[ques_vec]
    word_count_array = np.empty((len(topic_vec), 2), dtype = np.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = np.argsort(word_count_array[:, 1])
    idx = idx[::-1]
    word_count_array = word_count_array[idx]
    final = []
    final = ldamodel_2017.print_topic(word_count_array[0, 0], 100)
    #print(final)
    question_topic = final.split('*') ## as format is like "probability * topic"
    tokens = ' '.join(re.findall(r"[\w']+", str(final))).lower().split()
    x = [''.join(c for c in s if c not in string.punctuation) for s in tokens]
    #print(x)
    result = ' '.join([i for i in x if not i.isdigit()])
    #x=' '.join(x)
    print("Original Query: ",question)
    print("(Topic, Probability): ",ldamodel_2017.get_document_topics(ques_vec))
    #resword=ldamodel_2017.print_topic(ldamodel_2017.get_document_topics(ques_vec)[0][0])
    #print("Relevant Words:\n ")
    #display(ldamodel_2017.print_topic(ldamodel_2017.get_document_topics(ques_vec)[0][0]))
    return result.split()[0:9]

In [13]:
topicwords={}
#ldamodel.show_topics(30,10)
#print(topicwords)
no=0
for prob in ldamodel.show_topics(30,10):
    #print(prob)
    #print(prob[1])
    tokens = ' '.join(re.findall(r"[\w']+", str(prob[1]))).lower().split()
    x = [''.join(c for c in s if c not in string.punctuation) for s in tokens]
    result = ' '.join([i for i in x if not i.isdigit()])
    topicwords[no]=result.split()
    no+=1
for i in range(30):
    print("Topic",i,": ",topicwords[i])
#def gettopicwords_2017()

Topic 0 :  ['year', 'daughter', 'today', 'face', 'wonderful', 'whats', 'thing', 'time', 'rash', 'little']
Topic 1 :  ['test', 'blood', 'pain', 'anyone', 'work', 'positive', 'negative', 'doctor', 'month', 'long']
Topic 2 :  ['thank', 'time', 'uveitis', 'infusion', 'help', 'week', 'work', 'many', 'cause', 'something']
Topic 3 :  ['help', 'doctor', 'foot', 'time', 'psoriasis', 'work', 'thing', 'heat', 'thankful', 'dermatologist']
Topic 4 :  ['people', 'psoriasis', 'understand', 'life', 'help', 'many', 'others', 'disease', 'talk', 'someone']
Topic 5 :  ['life', 'psoriasis', 'arthritis', 'help', 'psoriatic', 'year', 'live', 'nice', 'disease', 'share']
Topic 6 :  ['bless', 'amen', 'jesus', 'name', 'heal', 'little', 'pray', 'prayer', 'child', 'lord']
Topic 7 :  ['psoriasis', 'group', 'help', 'scalp', 'treatment', 'information', 'year', 'hair', 'work', 'support']
Topic 8 :  ['pain', 'joint', 'hand', 'year', 'flare', 'finger', 'cant', 'last', 'knee', 'foot']
Topic 9 :  ['year', 'diagnose', 'sta

In [15]:
topicwords_2017={}
#ldamodel.show_topics(30,10)
#print(topicwords)
no=0
for prob in ldamodel_2017.show_topics(20,10):
    #print(prob)
    #print(prob[1])
    tokens = ' '.join(re.findall(r"[\w']+", str(prob[1]))).lower().split()
    x = [''.join(c for c in s if c not in string.punctuation) for s in tokens]
    result = ' '.join([i for i in x if not i.isdigit()])
    topicwords_2017[no]=result.split()
    no+=1
for i in range(20):
    print("Topic",i,": ",topicwords_2017[i])
#def gettopicwords_2017()

Topic 0 :  ['psoriasis', 'feel', 'time', 'need', 'team', 'cure', 'people', 'cant', 'work', 'pain']
Topic 1 :  ['psoriasis', 'anda', 'people', 'yang', 'time', 'nail', 'anyone', 'diagnose', 'boleh', 'saya']
Topic 2 :  ['help', 'pain', 'work', 'bless', 'family', 'little', 'sleep', 'time', 'thing', 'psoriasis']
Topic 3 :  ['pain', 'psoriasis', 'year', 'many', 'help', 'start', 'know', 'stress', 'little', 'time']
Topic 4 :  ['pain', 'help', 'good', 'work', 'cure', 'anyone', 'life', 'thing', 'hand', 'flare']
Topic 5 :  ['psoriasis', 'food', 'good', 'help', 'skin', 'thank', 'diet', 'free', 'live', 'blood']
Topic 6 :  ['year', 'psoriasis', 'people', 'group', 'help', 'week', 'joint', 'bless', 'please', 'tell']
Topic 7 :  ['psoriasis', 'anyone', 'mine', 'test', 'diagnose', 'work', 'thank', 'good', 'clear', 'time']
Topic 8 :  ['work', 'year', 'week', 'good', 'effect', 'month', 'methotrexate', 'side', 'life', 'humira']
Topic 9 :  ['psoriasis', 'help', 'treatment', 'disease', 'skin', 'people', 'time

In [14]:
question="I feel painful, psoriasis suffers a lot. Can someone help me"
getTopicForQuery_2017(question)

Original Query:  I feel painful, psoriasis suffers a lot. Can someone help me
(Topic, Probability):  [(15, 0.84166666358769671)]


['psoriasis', 'life', 'cancer', 'pain', 'live', 'help', 'skin', 'cure', 'time']